In [3]:
import openai
import pandas as pd
import time
import os
from dotenv import load_dotenv
import openai
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [7]:
output_dir = "processed_output/"
#Load the Parquet file into a DataFrame
df_qa = pd.read_parquet(os.path.join(output_dir, "extracted_data_qa_final.parquet"))
# Check the loaded DataFrame
df_qa

,question_type,question,answer,options
0,multiple_choice,1) Inertia of a body depends on,c) mass of the object,"{'a': 'weight of the object', 'b': 'accelerati..."
1,multiple_choice,2) Impulse is equals to,c) change of momentum,"{'a': 'rate of change of momentum', 'b': 'rate..."
2,multiple_choice,3) Newton's III law is applicable,c) both a & b,"{'a': 'for a body is at rest', 'b': 'for a bod..."
3,multiple_choice,4) Plotting a graph for momentum on the Y-axis...,c) Force,"{'a': 'Impulsive force', 'b': 'Acceleration', ..."
4,multiple_choice,5) In which of the following sport the turning...,d) hockey,"{'a': 'swimming', 'b': 'tennis', 'c': 'cycling..."
5,multiple_choice,6) The unit of ‘g’ is m/s². It can be also exp...,b) N/kg,"{'a': 'cm/s²', 'b': 'N/kg', 'c': 'Nm/kg', 'd':..."
6,multiple_choice,7) One kilogram force equals to,b) 9.8 x 10³ N,"{'a': '9.8 dyne', 'b': '9.8 x 10³ N', 'c': '98..."
7,multiple_choice,8) The mass of a body is measured on planet Ea...,a) 4M,"{'a': '4M', 'b': '2M', 'c': 'M/4', 'd': 'M'}"
8,multiple_choice,9) If the Earth shrinks to 50% of its real rad...,d) increase by 300%,"{'a': 'decrease by 50%', 'b': 'increase by 50%..."
9,multiple_choice,10) To project the rockets which of the follow...,d) both a and c,"{'a': 'Newton's third law of motion', 'b': 'Ne..."


In [10]:
# Set up OpenAI client with your API key
client = openai.OpenAI(api_key=openai_api_key)

# Function to get summary from OpenAI API
def get_summary(row):
    try:
        question_type = row['question_type']
        question      = row['question']
        options       = row['options']
    
        if question_type == 'multiple_choice':
            text = f"Context:{'None'}, Question type:{question_type}, Question:{question}, \nOptions:{options}"
        else:
            text = f"Context:{'None'}, Question type:{question_type}, Question:{question}"
        
        response = client.chat.completions.create(
            model="gpt-4",  # Use "gpt-3.5-turbo" if needed
            messages=[
                {"role": "system", "content": "please answer the question precisely ."},
                {"role": "user", "content": text}
            ],
            temperature=0.5,
            max_tokens=1024
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error: {e}")
        return None

df_qa["answer"] = df_qa.apply(get_summary, axis=1)
df_qa.to_parquet(os.path.join(output_dir, "extracted_data_summary_qa_openai_plain_answer.parquet"), index=False)
df_qa

,question_type,question,answer,options
0,multiple_choice,1) Inertia of a body depends on,'c': 'mass of the object',"{'a': 'weight of the object', 'b': 'accelerati..."
1,multiple_choice,2) Impulse is equals to,'c': 'change of momentum',"{'a': 'rate of change of momentum', 'b': 'rate..."
2,multiple_choice,3) Newton's III law is applicable,'c': 'both a & b',"{'a': 'for a body is at rest', 'b': 'for a bod..."
3,multiple_choice,4) Plotting a graph for momentum on the Y-axis...,'a': 'Impulsive force',"{'a': 'Impulsive force', 'b': 'Acceleration', ..."
4,multiple_choice,5) In which of the following sport the turning...,b: 'tennis',"{'a': 'swimming', 'b': 'tennis', 'c': 'cycling..."
5,multiple_choice,6) The unit of ‘g’ is m/s². It can be also exp...,'b': 'N/kg',"{'a': 'cm/s²', 'b': 'N/kg', 'c': 'Nm/kg', 'd':..."
6,multiple_choice,7) One kilogram force equals to,'c': '98 x 10⁴ dyne',"{'a': '9.8 dyne', 'b': '9.8 x 10³ N', 'c': '98..."
7,multiple_choice,8) The mass of a body is measured on planet Ea...,'d': 'M',"{'a': '4M', 'b': '2M', 'c': 'M/4', 'd': 'M'}"
8,multiple_choice,9) If the Earth shrinks to 50% of its real rad...,'d': 'increase by 300%',"{'a': 'decrease by 50%', 'b': 'increase by 50%..."
9,multiple_choice,10) To project the rockets which of the follow...,'d': 'both a and c',"{'a': 'Newton's third law of motion', 'b': 'Ne..."
